# Optimized for Maximal Sharpe Ratio Stock Portfolio

**Disclaimer:** _Investing in the stock market involves risk and can lead to monetary loss. This material is purely for educational purposes and should not be taken as professional investment advice. Invest at your own discretion._

Based on [Build A Killer Stock Portfolio Using Python](https://www.youtube.com/watch?v=bvDkel5whUY) of channel [Computer Science](https://www.youtube.com/channel/UCbmb5IoBtHZTpYZCDBOC1CA).

Description: This program optimizes a stock portfolio for maximal Sharpe Ratio using Efficient Frontier approach.

Required Python Packages: pandas, numpy, requests, pulp, PyPortfolioOpt

In [1]:
import pandas as pd
import numpy as np
import requests

Load the data

In [2]:
df = pd.read_csv('NYSE_close_outer.csv')

In [3]:
df

,Date,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AEP,...,TXN,TCOM,VRSN,VRSK,VRTX,WBA,WDAY,XEL,XLNX,ZM
0,2014-01-28,15.818016,59.110001,3.540000,133.009995,57.470001,562.067078,559.408203,394.429993,37.132782,...,35.605633,20.129999,59.599998,63.016399,79.300003,47.759052,87.589996,22.504726,39.903522,NaN
1,2014-01-29,16.026522,58.459999,3.480000,133.729996,57.740002,554.014038,551.393250,384.200012,37.195316,...,35.438469,19.570000,59.009998,62.838783,78.820000,47.323521,86.199997,22.472805,39.834736,NaN
2,2014-01-30,16.358236,59.389999,3.480000,162.000000,58.490002,568.263245,565.575073,403.010010,37.703342,...,35.680847,19.805000,59.439999,63.697292,82.150002,47.960075,89.769997,22.927694,40.221642,NaN
3,2014-01-31,16.235027,59.189999,3.430000,158.729996,59.419998,591.076050,588.279968,358.690002,38.148834,...,35.438469,19.754999,58.750000,63.016399,79.040001,48.035450,89.540001,23.071331,39.912117,NaN
4,2014-02-03,16.168686,58.090000,3.330000,154.669998,54.959999,567.282288,564.598755,346.149994,37.758053,...,34.686230,18.584999,56.279999,61.950668,77.629997,46.544556,87.940002,22.831926,38.708393,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,2021-01-21,94.599998,472.019989,91.529999,157.860001,543.809998,1884.150024,1891.250000,3306.989990,80.970001,...,175.100006,33.689999,197.139999,194.869995,238.639999,47.070000,229.389999,65.110001,141.160004,382.809998
1760,2021-01-22,94.430000,472.440002,92.790001,159.729996,534.080017,1892.560059,1901.050049,3292.229980,82.080002,...,172.809998,33.580002,194.750000,192.759995,237.589996,47.450001,235.529999,64.760002,142.940002,383.399994
1761,2021-01-25,93.400002,473.440002,94.129997,161.320007,539.469971,1894.280029,1899.400024,3294.000000,82.589996,...,172.919998,32.779999,192.000000,188.679993,241.309998,47.820000,234.990005,66.500000,145.139999,391.829987
1762,2021-01-26,92.379997,476.279999,94.709999,161.750000,535.099976,1907.949951,1917.239990,3326.129883,81.820000,...,171.470001,31.969999,196.050003,188.500000,237.639999,49.189999,235.600006,65.809998,144.389999,374.390015


Set the date as the index

In [4]:
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

In [5]:
df

,Date,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AEP,...,TXN,TCOM,VRSN,VRSK,VRTX,WBA,WDAY,XEL,XLNX,ZM
2014-01-28,2014-01-28,15.818016,59.110001,3.540000,133.009995,57.470001,562.067078,559.408203,394.429993,37.132782,...,35.605633,20.129999,59.599998,63.016399,79.300003,47.759052,87.589996,22.504726,39.903522,NaN
2014-01-29,2014-01-29,16.026522,58.459999,3.480000,133.729996,57.740002,554.014038,551.393250,384.200012,37.195316,...,35.438469,19.570000,59.009998,62.838783,78.820000,47.323521,86.199997,22.472805,39.834736,NaN
2014-01-30,2014-01-30,16.358236,59.389999,3.480000,162.000000,58.490002,568.263245,565.575073,403.010010,37.703342,...,35.680847,19.805000,59.439999,63.697292,82.150002,47.960075,89.769997,22.927694,40.221642,NaN
2014-01-31,2014-01-31,16.235027,59.189999,3.430000,158.729996,59.419998,591.076050,588.279968,358.690002,38.148834,...,35.438469,19.754999,58.750000,63.016399,79.040001,48.035450,89.540001,23.071331,39.912117,NaN
2014-02-03,2014-02-03,16.168686,58.090000,3.330000,154.669998,54.959999,567.282288,564.598755,346.149994,37.758053,...,34.686230,18.584999,56.279999,61.950668,77.629997,46.544556,87.940002,22.831926,38.708393,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-21,2021-01-21,94.599998,472.019989,91.529999,157.860001,543.809998,1884.150024,1891.250000,3306.989990,80.970001,...,175.100006,33.689999,197.139999,194.869995,238.639999,47.070000,229.389999,65.110001,141.160004,382.809998
2021-01-22,2021-01-22,94.430000,472.440002,92.790001,159.729996,534.080017,1892.560059,1901.050049,3292.229980,82.080002,...,172.809998,33.580002,194.750000,192.759995,237.589996,47.450001,235.529999,64.760002,142.940002,383.399994
2021-01-25,2021-01-25,93.400002,473.440002,94.129997,161.320007,539.469971,1894.280029,1899.400024,3294.000000,82.589996,...,172.919998,32.779999,192.000000,188.679993,241.309998,47.820000,234.990005,66.500000,145.139999,391.829987
2021-01-26,2021-01-26,92.379997,476.279999,94.709999,161.750000,535.099976,1907.949951,1917.239990,3326.129883,81.820000,...,171.470001,31.969999,196.050003,188.500000,237.639999,49.189999,235.600006,65.809998,144.389999,374.390015


Remove the Date columns

In [6]:
df.drop(columns=['Date'], axis=1, inplace=True)

In [7]:
df

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AEP,AMGN,...,TXN,TCOM,VRSN,VRSK,VRTX,WBA,WDAY,XEL,XLNX,ZM
2014-01-28,15.818016,59.110001,3.540000,133.009995,57.470001,562.067078,559.408203,394.429993,37.132782,100.930412,...,35.605633,20.129999,59.599998,63.016399,79.300003,47.759052,87.589996,22.504726,39.903522,NaN
2014-01-29,16.026522,58.459999,3.480000,133.729996,57.740002,554.014038,551.393250,384.200012,37.195316,99.651031,...,35.438469,19.570000,59.009998,62.838783,78.820000,47.323521,86.199997,22.472805,39.834736,NaN
2014-01-30,16.358236,59.389999,3.480000,162.000000,58.490002,568.263245,565.575073,403.010010,37.703342,101.089317,...,35.680847,19.805000,59.439999,63.697292,82.150002,47.960075,89.769997,22.927694,40.221642,NaN
2014-01-31,16.235027,59.189999,3.430000,158.729996,59.419998,591.076050,588.279968,358.690002,38.148834,99.467049,...,35.438469,19.754999,58.750000,63.016399,79.040001,48.035450,89.540001,23.071331,39.912117,NaN
2014-02-03,16.168686,58.090000,3.330000,154.669998,54.959999,567.282288,564.598755,346.149994,37.758053,97.627403,...,34.686230,18.584999,56.279999,61.950668,77.629997,46.544556,87.940002,22.831926,38.708393,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-21,94.599998,472.019989,91.529999,157.860001,543.809998,1884.150024,1891.250000,3306.989990,80.970001,252.029999,...,175.100006,33.689999,197.139999,194.869995,238.639999,47.070000,229.389999,65.110001,141.160004,382.809998
2021-01-22,94.430000,472.440002,92.790001,159.729996,534.080017,1892.560059,1901.050049,3292.229980,82.080002,253.500000,...,172.809998,33.580002,194.750000,192.759995,237.589996,47.450001,235.529999,64.760002,142.940002,383.399994
2021-01-25,93.400002,473.440002,94.129997,161.320007,539.469971,1894.280029,1899.400024,3294.000000,82.589996,257.100006,...,172.919998,32.779999,192.000000,188.679993,241.309998,47.820000,234.990005,66.500000,145.139999,391.829987
2021-01-26,92.379997,476.279999,94.709999,161.750000,535.099976,1907.949951,1917.239990,3326.129883,81.820000,258.600006,...,171.470001,31.969999,196.050003,188.500000,237.639999,49.189999,235.600006,65.809998,144.389999,374.390015


 Get the assets / tickers

In [8]:
assets = df.columns

In [9]:
# !pip3 install PyPortfolioOpt

Optimize the portfolio

In [10]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns

In [11]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

/home/vladiant/.local/lib/python3.8/site-packages/pypfopt/risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


Optimize for the maximal Sharpe Ratio

In [12]:
# Create the Efficient Frontier object
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

In [13]:
cleaned_weights = ef.clean_weights()
cleaned_weights

OrderedDict([('ATVI', 0.0),
             ('ADBE', 0.0),
             ('AMD', 0.0),
             ('ALXN', 0.0),
             ('ALGN', 0.0),
             ('GOOGL', 0.0),
             ('GOOG', 0.0),
             ('AMZN', 0.0),
             ('AEP', 0.0),
             ('AMGN', 0.0),
             ('ADI', 0.0),
             ('ANSS', 0.0),
             ('AAPL', 0.0),
             ('AMAT', 0.0),
             ('ASML', 0.0),
             ('TEAM', 0.0),
             ('ADSK', 0.0),
             ('ADP', 0.0),
             ('BIDU', 0.0),
             ('BIIB', 0.0),
             ('BKNG', 0.0),
             ('AVGO', 0.0),
             ('CDNS', 0.0),
             ('CDW', 0.0),
             ('CERN', 0.0),
             ('CHTR', 0.0),
             ('CHKP', 0.0),
             ('CTAS', 0.08844),
             ('CSCO', 0.0),
             ('CTSH', 0.0),
             ('CMCSA', 0.0),
             ('CPRT', 0.0),
             ('COST', 0.0),
             ('CSX', 0.0),
             ('DXCM', 0.0),
             ('DOCU'

In [14]:
ef.portfolio_performance(verbose=True)

Expected annual return: 130.0%
Annual volatility: 28.2%
Sharpe Ratio: 4.53


(1.3003857043418665, 0.282496925034705, 4.53238811071862)

Sharpe Ratio > 1.0 is considered acceptable to good

In [15]:
# !pip3 install pulp

Get the discrete allocation of each share per stock

In [16]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [17]:
portfolio_val = 5000 # Money to put in portfolio USD 

In [18]:
lastest_prices = get_latest_prices(df)

In [19]:
weights = cleaned_weights
da = DiscreteAllocation(weights, lastest_prices, total_portfolio_value=portfolio_val)
allocation, leftover = da.lp_portfolio()

In [20]:
print('Discrete allocation:', allocation)
print('Funds Remaining: $', leftover)

Discrete allocation: {'CTAS': 2, 'KDP': 5, 'MAR': 8, 'MRNA': 4, 'PTON': 10, 'PDD': 1, 'ROST': 6, 'ZM': 1}
Funds Remaining: $ 0.9399223327636719


Create a function to get the company name

In [21]:
def get_company_name(symbol):
    url = 'http://d.yimg.com/autoc.finance.yahoo.com/autoc?query='+symbol+'&region=1&lang=en'
    result = requests.get(url).json()
    for r in result['ResultSet']['Result']:
        if r['symbol']==symbol:
            return r['name']

Store the company name into a list

In [22]:
company_name = []
for symbol in allocation:
    company_name.append(get_company_name(symbol))

Get the discrete allocation values

In [23]:
discrete_allocaion_list = []
for symbol in allocation:
    discrete_allocaion_list.append(allocation.get(symbol))

Create a dataframe for the portfolio

In [24]:
portfolio_df = pd.DataFrame(columns=['Company_name', 'Company_ticker', 'Discrete_val_'+str(portfolio_val)])

In [25]:
portfolio_df['Company_name'] = company_name
portfolio_df['Company_ticker'] = allocation
portfolio_df['Discrete_val_'+str(portfolio_val)] = discrete_allocaion_list

Show the portfolio

**THIS IS NOT AN INVESTMENT ADVICE!**

In [26]:
portfolio_df

,Company_name,Company_ticker,Discrete_val_5000
0,Cintas Corporation,CTAS,2
1,Keurig Dr Pepper Inc.,KDP,5
2,"Marriott International, Inc.",MAR,8
3,"Moderna, Inc.",MRNA,4
4,"Peloton Interactive, Inc.",PTON,10
5,Pinduoduo Inc.,PDD,1
6,"Ross Stores, Inc.",ROST,6
7,"Zoom Video Communications, Inc.",ZM,1
